In [ ]:
# import numpy and matplotlib
import numpy as np
import matplotlib.pyplot as plt

# import mini MD modules
from miniMD import models
from miniMD import samplers
# import function to compute autocorrelation
from miniMD.samplers import autocorr

## Exercise 2.2:
In this exercise we will explore how the Ensemble Quasi Newton introduced in the lecture (see also 
https://link.springer.com/article/10.1007/s11222-017-9730-1) can improve the sampling in the case of not well conditioned target distributions. We will focus on the case of a Gaussian target density, i.e.,  

$$
\rho_{target}(x) \propto e^{- \frac{1}{2}x^{T}\Omega x}
$$

where the matrix $\Omega$ has a large condition number. Such a target density can be specified using the code below:



In [ ]:
#Specify precision matrix of multivariate Gaussian target density 
M = 100.0
Omega = np.array([[1/M,0.0],
                  [0.0,1.0]])
#Specify initial condition 
q_0 = np.array([0.0,0.0])
p_0 = np.array([0.0,0.0])
model = models.HarmonicOscillatorMultiDim(q=q_0,
                                    p=p_0, Omega=Omega)

A version of the Ensemble Quasi Newton method with non-local covariance estimates is implemented as EnsembleQuasiNewton as a subclass of the class UnderdampedLangevinSampler within the module miniMD.sampler. This sampler can be initialized by providing a collection of copies of the model to be sampled and a the usual parameters for Langevin samplers:

In [ ]:
eqn_sampler = samplers.EnsembleQuasiNewton(repmodel=replicated_model, 
                         stepsize=.1, 
                         inverse_temperature=1.0,
                         friction_constant=1.0, 
                        ) 

The above code probably returned an error message since we haven't specified the object "replicated_model" yet. The argument "repmodel" in the initialisation of "EnsembleQuasiNewton" is required to be an instance of a subclass of "models.ReplicatedModel". We can create an instance of "models.ReplicatedModel" using the code below

In [ ]:
replicated_model = models.ReplicatedModel(model, # model specifying target density 
                                          nreplicas=1 # number of replicas/copies to be created
                                         )

After initialising "eqn_sampler" you can use this object to sample from your target density, e.g.,

In [ ]:
q_trajectory, p_trajectory = eqn_sampler.sample(nsteps=10000) # sample the system

### Task 1

For fixed stepsize $\Delta t = .1$ compare the performance of a standard BAOAB integrator (Hint: for nreplicas=1, the EnsembleQuasiNewton method is identical to standard BAOAB) and the Ensemble Quasi Newton method for the 2-dimensional Multivariate Gaussian specified above by inspecting trace plots of $x_1$ and $x_2$ and/or the autocorrelation function of these variables.
-  vary the condition number of $\Omega$ by changing the value of $M$ (Hint: better only increase M, so that you don't run into stability issues). How does this affect the performance of standard BAOAB and the Ensemble Quasi Newton method, respectively?
- How sensitive is the performance of the Ensemble Quasi Newton method on the number of replicas? 
- __Optional__: You can experiment with changing the frequency at which the preconditioning matrix is updated by modifying the optional argument "B_update_mod" when initializing the EQN sampler (The deault value is B_update_mod=1). 